# Statistical and Machine Learning Models for Fundamentalist Data

This notebook is a useful tool for investors interested in the Brazilian stock market. It integrates machine learning techniques and statistical models to analyze fundamentalist data of companies listed on the stock exchange. The aim is to provide in-depth analysis and facilitate investment decision-making, focusing on identifying opportunities and mitigating risks. It includes interactive visualizations and real-time updates, making it accessible and practical for both experienced investors and beginners.

## Initial Setup

### Install Packages

In [408]:
%pip install pandas -q
%pip install plotly -q
%pip install scikit-learn -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import libs

In [580]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score as sil_score
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

### Create a file path default

In [410]:
file_path_book = str(Path(os.getcwd()).parent.parent / "data/book")

### Load data

In [411]:
df_fundamentals_book = pd.read_csv(file_path_book + "/fundamentals_book.csv")
df_fundamentals_book.head(5)

,ticker,long_name,sector,industry,market_cap,enterprise_value,total_revenue,profit_margins,operating_margins,dividend_rate,beta,ebitda,trailing_pe,forward_pe,volume,average_volume,fifty_two_week_low,fifty_two_week_high,price_to_sales_trailing_12_months,fifty_day_average,two_hundred_day_average,trailing_annual_dividend_rate,trailing_annual_dividend_yield,book_value,price_to_book,total_cash,total_cash_per_share,total_debt,earnings_quarterly_growth,revenue_growth,gross_margins,ebitda_margins,return_on_assets,return_on_equity,gross_profits,total_assets_approx,asset_turnover,earnings_growth_rate,dividend_payout_ratio,equity,debt_to_equity,roi,roce
0,ABCB4.SA,Banco ABC Brasil S.A.,Financial Services,Banks - Regional,4.265434e+09,1.477339e+10,1.941779e+09,0.41576,0.38826,1.56,0.679,0.000000e+00,4.069768,4.706601,92300.0,747165.0,15.85,21.99,2.196663,19.3382,18.14667,1.550,0.080687,24.518,0.785138,7.774306e+09,35.162,1.829846e+10,0.001,0.003,0.00000,0.00000,0.01530,0.15680,1.973086e+09,7.774306e+09,0.249769,0.1,155000.000000,-1.052416e+10,-1.738710,0.131438,0.000000
1,AGRO3.SA,BrasilAgro - Companhia Brasileira de Proprieda...,Consumer Defensive,Farm Products,2.466480e+09,2.912933e+09,1.249437e+09,0.21493,0.25031,3.21,0.432,2.648920e+08,9.450382,6.332481,298100.0,666692.0,22.29,32.71,1.974073,27.0106,25.58635,3.240,0.132029,22.237,1.113460,3.838370e+08,3.885,8.720750e+08,6.801,0.671,0.25252,0.21201,0.03839,0.12170,3.155040e+08,3.838370e+08,3.255124,680.1,47.640053,-4.882380e+08,-1.786168,0.428927,0.079343
2,RAIL3.SA,Rumo S.A.,Industrials,Railroads,4.228882e+10,5.524305e+10,1.031746e+10,0.07639,0.33544,0.07,0.227,4.522541e+09,54.309525,21.723810,5733400.0,14644522.0,16.21,24.44,4.098764,22.5852,20.95235,0.066,0.002993,8.334,2.736981,7.656040e+09,4.132,2.184320e+10,3.935,0.121,0.34493,0.43834,0.04252,0.05163,3.146360e+09,7.656040e+09,1.347623,393.5,1.677255,-1.418716e+10,-1.539646,0.186765,0.070519
3,ALPA3.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.309793e+09,6.482982e+09,4.022153e+09,-0.05671,-0.06434,0.40,0.571,-1.980000e+05,0.000000,0.000000,1100.0,3953.0,7.27,17.80,1.320137,8.7146,9.63540,0.000,0.000000,7.867,1.008008,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.620417,-0.000029
4,ALPA4.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.350758e+09,6.395236e+09,4.022153e+09,-0.05671,-0.06434,0.43,0.571,-1.980000e+05,0.000000,14.555555,1132100.0,5605825.0,6.81,22.51,1.330322,8.3228,9.27290,0.000,0.000000,7.867,0.999110,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.628930,-0.000029


## Models

#### Data normalization using StandardScaler

In [422]:
numeric_columns = df_fundamentals_book.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df_fundamentals_book.drop(['ticker', 'long_name', 'industry'], axis='columns').copy(deep=True)
categorical_columns = categorical_columns.select_dtypes(include='object').columns

#### Transforming categorical features into numerical using OneHotEncoder

In [423]:
encoder = OneHotEncoder(sparse=False)
encoded_categorical = encoder.fit_transform(df_fundamentals_book[categorical_columns])

df_encoded = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))
df_fundamentals_final = pd.concat([df_fundamentals_book.drop(categorical_columns, axis=1), df_encoded], axis=1)
df_fundamentals_final.columns = ['_'.join(col.lower().replace('-', '').split()) for col in df_fundamentals_final.columns]

df_fundamentals_final.head()

,ticker,long_name,industry,market_cap,enterprise_value,total_revenue,profit_margins,operating_margins,dividend_rate,beta,ebitda,trailing_pe,forward_pe,volume,average_volume,fifty_two_week_low,fifty_two_week_high,price_to_sales_trailing_12_months,fifty_day_average,two_hundred_day_average,trailing_annual_dividend_rate,trailing_annual_dividend_yield,book_value,price_to_book,total_cash,total_cash_per_share,total_debt,earnings_quarterly_growth,revenue_growth,gross_margins,ebitda_margins,return_on_assets,return_on_equity,gross_profits,total_assets_approx,asset_turnover,earnings_growth_rate,dividend_payout_ratio,equity,debt_to_equity,roi,roce,sector_basic_materials,sector_communication_services,sector_consumer_cyclical,sector_consumer_defensive,sector_energy,sector_financial_services,sector_healthcare,sector_industrials,sector_real_estate,sector_technology,sector_utilities
0,ABCB4.SA,Banco ABC Brasil S.A.,Banks - Regional,4.265434e+09,1.477339e+10,1.941779e+09,0.41576,0.38826,1.56,0.679,0.000000e+00,4.069768,4.706601,92300.0,747165.0,15.85,21.99,2.196663,19.3382,18.14667,1.550,0.080687,24.518,0.785138,7.774306e+09,35.162,1.829846e+10,0.001,0.003,0.00000,0.00000,0.01530,0.15680,1.973086e+09,7.774306e+09,0.249769,0.1,155000.000000,-1.052416e+10,-1.738710,0.131438,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,AGRO3.SA,BrasilAgro - Companhia Brasileira de Proprieda...,Farm Products,2.466480e+09,2.912933e+09,1.249437e+09,0.21493,0.25031,3.21,0.432,2.648920e+08,9.450382,6.332481,298100.0,666692.0,22.29,32.71,1.974073,27.0106,25.58635,3.240,0.132029,22.237,1.113460,3.838370e+08,3.885,8.720750e+08,6.801,0.671,0.25252,0.21201,0.03839,0.12170,3.155040e+08,3.838370e+08,3.255124,680.1,47.640053,-4.882380e+08,-1.786168,0.428927,0.079343,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,RAIL3.SA,Rumo S.A.,Railroads,4.228882e+10,5.524305e+10,1.031746e+10,0.07639,0.33544,0.07,0.227,4.522541e+09,54.309525,21.723810,5733400.0,14644522.0,16.21,24.44,4.098764,22.5852,20.95235,0.066,0.002993,8.334,2.736981,7.656040e+09,4.132,2.184320e+10,3.935,0.121,0.34493,0.43834,0.04252,0.05163,3.146360e+09,7.656040e+09,1.347623,393.5,1.677255,-1.418716e+10,-1.539646,0.186765,0.070519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,ALPA3.SA,Alpargatas S.A.,Footwear & Accessories,5.309793e+09,6.482982e+09,4.022153e+09,-0.05671,-0.06434,0.40,0.571,-1.980000e+05,0.000000,0.000000,1100.0,3953.0,7.27,17.80,1.320137,8.7146,9.63540,0.000,0.000000,7.867,1.008008,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.620417,-0.000029,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ALPA4.SA,Alpargatas S.A.,Footwear & Accessories,5.350758e+09,6.395236e+09,4.022153e+09,-0.05671,-0.06434,0.43,0.571,-1.980000e+05,0.000000,14.555555,1132100.0,5605825.0,6.81,22.51,1.330322,8.3228,9.27290,0.000,0.000000,7.867,0.999110,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.628930,-0.000029,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Feature Selection using PCA

In [577]:
n_components = 5
pca = PCA(n_components)
pca_result = pca.fit_transform(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))
explained_variance = pca.explained_variance_ratio_

fig = px.bar(x=[f'PC{i+1}' for i in range(len(explained_variance))], y=explained_variance, labels={'x': 'Principal Component', 'y': 'Explained Variance'}, title='Explained Variance by Each Principal Component', color_discrete_sequence=['rgb(100, 195, 181)'], text=[f'{x:.2f}%' for x in explained_variance*100])
fig.update_traces(textposition='outside')
fig.update_layout(template='plotly_dark', font=dict(color='white'), height=500)
fig.show()

##### Visualizing the fields within each principal component.

In [443]:
pca = PCA(n_components=10)
pca.fit(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))
variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(variance_explained) + 1), y=cumulative_variance, mode='lines+markers', line=dict(color='rgb(100, 195, 181)'), name='Cumulative Variance Explained'))
fig.update_layout(title='Elbow Method for PCA Analysis', xaxis_title='Number of Principal Components', yaxis_title='Cumulative Variance Explained', xaxis=dict(tickmode='linear'), yaxis=dict(tickformat='.0%'), template='plotly_dark', font=dict(color='white')) 
fig.show()

In [444]:
pca = PCA(n_components=4)
principal_components = pca.fit_transform(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))
range_num_clusters = list(range(2, 10))
silhouette_scores = []
distortions = []

for num_cluster in range_num_clusters:

    kmeans = KMeans(n_clusters=num_cluster, max_iter=10_000, random_state=19051992)
    cluster_labels = kmeans.fit_predict(principal_components)
    silhouette_scores.append(sil_score(principal_components, cluster_labels))

    distortions.append(KMeans(n_clusters=num_cluster, max_iter=10_000, random_state=19051992).fit(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number'])).inertia_)

scaler = StandardScaler()
silhouette_scores_scaled = scaler.fit_transform(np.array(silhouette_scores).reshape(-1, 1)).flatten()
distortions_scaled = scaler.fit_transform(np.array(distortions).reshape(-1, 1)).flatten()

In [578]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=range_num_clusters, y=silhouette_scores_scaled, mode='lines+markers', name='Silhouette Score'))
fig.add_trace(go.Scatter(x=range_num_clusters, y=distortions_scaled, mode='lines+markers', name='Distortions'))
fig.update_layout(title='Elbow Method for Determining the Optimal Number of Clusters (Normalized)', xaxis_title='Number of clusters', yaxis_title='Silhouette Score / Distortions (Normalized)', template='plotly_dark', font=dict(color='white'))
fig.show()

In [589]:
df_pca = pd.DataFrame(principal_components[:, :2], columns=['PC1', 'PC2'])
df_pca['cluster'] = cluster_labels

fig = px.scatter(df_pca, x='PC1', y='PC2', color='cluster', title='Visualização de Clusters com PCA', color_continuous_scale=px.colors.qualitative.Vivid)
fig.update_layout(template='plotly_dark', font=dict(color='white'))
fig.show()


In [590]:
pca = PCA(n_components=5)
principal_components = pca.fit_transform(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))

tsne = TSNE(n_components=2, metric='cosine')
tsne_results = tsne.fit_transform(principal_components)

kmeans = KMeans(n_clusters=5, max_iter=10_000, random_state=19051992)
clusters = kmeans.fit_predict(tsne_results)

df_visualization = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
df_visualization['cluster'] = clusters

fig = px.scatter(df_visualization, x='TSNE1', y='TSNE2', color='cluster', title='Visualization of Clusters with TSNE and Cosine Distance', color_continuous_scale=px.colors.qualitative.Vivid)
fig.update_layout(template='plotly_dark', font=dict(color='white'))
fig.show()

In [592]:
df_fundamentals_final['kmeans_cluster'] = clusters
df_fundamentals_final['kmeans_cluster'].head()

0    1
1    2
2    0
3    2
4    2
Name: kmeans_cluster, dtype: int32